# Data Cleaning & Preprocessing

This notebook focuses on cleaning and preprocessing the raw financial data to prepare it for analysis.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add the src directory to the path
sys.path.append(os.path.abspath('../src'))

# Import custom modules
from data_loader import load_raw_data, clean_financial_data, save_cleaned_data

## 1. Load and Inspect Raw Data

In [ ]:
# Load raw data
raw_data_path = '../data/raw/financial_data.csv'
df_raw = load_raw_data(raw_data_path)

# Display basic information
print("\nDataset Shape:")
print(df_raw.shape)

print("\nColumn Names:")
print(df_raw.columns.tolist())

print("\nData Types:")
print(df_raw.dtypes)

print("\nFirst 5 rows:")
df_raw.head()

## 2. Check for Missing Values

In [ ]:
# Check for missing values
missing_values = df_raw.isnull().sum()
missing_percentage = (missing_values / len(df_raw)) * 100

missing_df = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percentage
})

print("Missing Values Summary:")
missing_df[missing_df['Missing Values'] > 0]

## 3. Check for Inconsistent Values

In [ ]:
# Check for placeholder values like '-', 'None', etc.
def check_placeholders(df, placeholders=['-', 'None', 'NaN', 'nan', 'NULL']):
    placeholder_counts = {}
    
    for col in df.columns:
        for placeholder in placeholders:
            count = (df[col] == placeholder).sum()
            if count > 0:
                if col not in placeholder_counts:
                    placeholder_counts[col] = {}
                placeholder_counts[col][placeholder] = count
    
    return placeholder_counts

placeholder_counts = check_placeholders(df_raw)
print("Placeholder Values Found:")
for col, counts in placeholder_counts.items():
    print(f"\n{col}:")
    for placeholder, count in counts.items():
        print(f"  {placeholder}: {count}")

In [ ]:
# Check for inconsistent categorical values
categorical_columns = df_raw.select_dtypes(include=['object']).columns

print("\nUnique values in categorical columns:")
for col in categorical_columns:
    print(f"\n{col}:")
    print(df_raw[col].value_counts().head(10))

In [ ]:
# Check for inconsistent currency formatting
def check_currency_format(df, columns):
    for col in columns:
        if col in df.columns:
            # Check for values with $ or commas
            has_dollar = df[col].astype(str).str.contains('\$').sum()
            has_comma = df[col].astype(str).str.contains(',').sum()
            
            print(f"\n{col}:")
            print(f"  Values with $ symbol: {has_dollar}")
            print(f"  Values with commas: {has_comma}")
            print(f"  Sample values: {df[col].astype(str).sample(5).tolist()}")

# Assuming Revenue, Cost, and Profit are currency columns
currency_columns = ['Revenue', 'Cost', 'Profit']
print("Currency formatting check:")
check_currency_format(df_raw, currency_columns)

In [ ]:
# Check for inconsistent date formatting
def check_date_format(df, columns):
    for col in columns:
        if col in df.columns:
            print(f"\n{col}:")
            print(f"  Sample values: {df[col].sample(10).tolist()}")

# Identify potential date columns
date_columns = [col for col in df_raw.columns if 'date' in col.lower()]
print("Date formatting check:")
check_date_format(df_raw, date_columns)

## 4. Clean the Data

In [ ]:
# Clean the data using our utility function
df_cleaned = clean_financial_data(df_raw)

# Display the cleaned data
print("Cleaned Data - First 5 rows:")
df_cleaned.head()

In [ ]:
# Compare before and after cleaning
print("Before cleaning:")
print(f"Shape: {df_raw.shape}")
print(f"Data types:\n{df_raw.dtypes}\n")

print("After cleaning:")
print(f"Shape: {df_cleaned.shape}")
print(f"Data types:\n{df_cleaned.dtypes}\n")

# Check for missing values after cleaning
missing_after = df_cleaned.isnull().sum()
print("Missing values after cleaning:")
print(missing_after[missing_after > 0])

## 5. Save Cleaned Data

In [ ]:
# Save the cleaned data
cleaned_data_path = '../data/processed/cleaned_data.csv'
save_cleaned_data(df_cleaned, cleaned_data_path)

## 6. Summary of Cleaning Process

In this notebook, we performed the following data cleaning steps:

1. **Loaded and inspected the raw data**
   - Examined the structure, columns, and data types
   - Identified potential issues

2. **Identified and handled missing values**
   - Detected explicit missing values (NaN)
   - Identified placeholder values like '-', 'None', etc.

3. **Fixed inconsistent formatting**
   - Standardized currency values by removing '$', commas, and whitespace
   - Standardized date formats to YYYY-MM-DD

4. **Normalized categorical variables**
   - Standardized inconsistent labels (e.g., 'Midmarket' vs 'Mid Market')

5. **Handled missing values**
   - Imputed numeric columns with median values
   - Imputed categorical columns with mode values

6. **Saved the cleaned dataset** for further analysis

The cleaned dataset is now ready for exploratory data analysis and modeling.